In [8]:
!pip install pymupdf


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   --- ------------------------------------ 1.6/18.7 MB 14.9 MB/s eta 0:00:02
   ------------- -------------------------- 6.6/18.7 MB 20.4 MB/s eta 0:00:01
   ------------------------ --------------- 11.5/18.7 MB 21.7 MB/s eta 0:00:01
   ------------------------------- -------- 14.9/18.7 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------  18.6/18.7 MB 20.6 MB/s eta 0:00:01
   ---------------------------------------- 18.7/18.7 MB 19.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import fitz

**Converting Pdf To Images**

In [11]:
import fitz  # PyMuPDF
import os

# List of your PDF files
pdf_files = ["COPGD835-51-1-MNP-2050 1.pdf", "COPGD835-51-1-MNP-2050.pdf"]

for pdf_path in pdf_files:
    doc = fitz.open(pdf_path)
    page = doc[0]  # first (and only) page
    pix = page.get_pixmap(dpi=300)

    # Get filename without extension
    base_name = os.path.splitext(pdf_path)[0]

    # Save image with same name as PDF
    image_name = f"{base_name}.png"
    pix.save(image_name)

    print(f"{pdf_path} converted to {image_name} successfully!")


COPGD835-51-1-MNP-2050 1.pdf converted to COPGD835-51-1-MNP-2050 1.png successfully!
COPGD835-51-1-MNP-2050.pdf converted to COPGD835-51-1-MNP-2050.png successfully!


In [15]:
import cv2
import pytesseract

# Point pytesseract to the installed tesseract.exe
pytesseract.pytesseract.tesseract_cmd = r"C:/Users/122328/AppData/Local/Programs/Tesseract-OCR"

img1_path = "COPGD835-51-1-MNP-2050 1.png"
img2_path = "COPGD835-51-1-MNP-2050.png"

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

def get_ocr_data(img):
    return pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

data1 = get_ocr_data(img1)
data2 = get_ocr_data(img2)


TesseractNotFoundError: C:/Users/122328/AppData/Local/Programs/Tesseract-OCR is not installed or it's not in your PATH. See README file for more information.

In [16]:
import fitz  # PyMuPDF
import os
import cv2
import pytesseract

# ==========================
# 1️⃣ Setup Tesseract path
# ==========================
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\122328\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# ==========================
# 2️⃣ List of PDFs
# ==========================
pdf_files = ["COPGD835-51-1-MNP-2050 1.pdf", "COPGD835-51-1-MNP-2050.pdf"]

# Convert PDFs to PNGs
image_paths = []
for pdf_path in pdf_files:
    doc = fitz.open(pdf_path)
    page = doc[0]  # first (only) page
    pix = page.get_pixmap(dpi=300)

    # Save PNG with same name as PDF
    base_name = os.path.splitext(pdf_path)[0]
    image_name = f"{base_name}.png"
    pix.save(image_name)
    image_paths.append(image_name)
    print(f"{pdf_path} converted to {image_name} successfully!")

# ==========================
# 3️⃣ Load images
# ==========================
img1 = cv2.imread(image_paths[0])
img2 = cv2.imread(image_paths[1])

# ==========================
# 4️⃣ OCR and get bounding boxes
# ==========================
def get_ocr_data(img):
    return pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

data1 = get_ocr_data(img1)
data2 = get_ocr_data(img2)

# ==========================
# 5️⃣ Compare words
# ==========================
# Clean and extract words
words1 = [w.strip() for w in data1["text"] if w.strip() != ""]
words2 = [w.strip() for w in data2["text"] if w.strip() != ""]

# Find differences (words that are unique in either file)
diff_words = set(words1) ^ set(words2)

# ==========================
# 6️⃣ Draw red rectangles for differences
# ==========================
# On first image
for i, word in enumerate(data1["text"]):
    if word.strip() != "" and word in diff_words:
        x, y, w, h = data1["left"][i], data1["top"][i], data1["width"][i], data1["height"][i]
        cv2.rectangle(img1, (x, y), (x + w, y + h), (0, 0, 255), 2)

# On second image
for i, word in enumerate(data2["text"]):
    if word.strip() != "" and word in diff_words:
        x, y, w, h = data2["left"][i], data2["top"][i], data2["width"][i], data2["height"][i]
        cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 0, 255), 2)

# ==========================
# 7️⃣ Save output images
# ==========================
cv2.imwrite("diff_file1.png", img1)
cv2.imwrite("diff_file2.png", img2)

print("Differences highlighted!")
print("Saved as diff_file1.png and diff_file2.png")


COPGD835-51-1-MNP-2050 1.pdf converted to COPGD835-51-1-MNP-2050 1.png successfully!
COPGD835-51-1-MNP-2050.pdf converted to COPGD835-51-1-MNP-2050.png successfully!
Differences highlighted!
Saved as diff_file1.png and diff_file2.png
